# Tidy3D mode solver

## Waveguides

In [ ]:
import gdsfactory.simulation.gtidy3d as gt

nm = 1e-3

In [ ]:
strip = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, wg_thickness=0.22, slab_thickness=0.0, ncore=gt.modes.si, nclad=gt.modes.sio2)
strip.plot_index()

In [ ]:
strip.plot_Ex(0)  # TE
strip.plot_Ey(1)  # TM

In [ ]:
rib = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, wg_thickness=0.22, slab_thickness=0.15, ncore=gt.modes.si, nclad=gt.modes.sio2)
rib.plot_index()

In [ ]:
rib.plot_Ex(mode_index=0)
rib.plot_Ey(mode_index=0)

In [ ]:
nitride = gt.modes.Waveguide(wavelength=1.55, wg_width=1.0, wg_thickness=0.4, slab_thickness=0.0, ncore=gt.modes.sin, nclad=gt.modes.sio2)
nitride.plot_index()
nitride.plot_Ex(0)
nitride.plot_Ey(0)

## Sweep width

You can sweep the waveguide width and compute the modes.

In [ ]:
df = gt.modes.sweep_width(
    width1=200*nm,
    width2=1000*nm,
    steps=11,
    wavelength=1.55,
    wg_thickness=220 * nm,
    slab_thickness=0 * nm,
    ncore=gt.modes.si,
    nclad=gt.modes.sio2,
)
gt.modes.plot_neff_vs_width(df)

## Bend modes

You can compute bend modes specifying the bend radius.

In [ ]:
strip_bend = gt.modes.Waveguide(
    wavelength=1.55, wg_width=0.5, wg_thickness=0.22, slab_thickness=0.0, bend_radius=3, ncore=gt.modes.si, nclad=gt.modes.sio2
)

In [ ]:
# lets plot the fundamental TE mode
strip_bend.plot_Ex(0)
strip_bend.plot_Ey(0)

In [ ]:
# lets plot the fundamental TM mode
strip_bend.plot_Ex(1)
strip_bend.plot_Ey(1)

## Bend loss

You can also compute the losses coming from the mode mismatch from the bend into a straight waveguide.
To compute the bend loss due to mode mismatch you can calculate the mode overlap of the straight mode and the bent mode.
Because there are two mode mismatch interfaces the total loss due to mode mismatch will be squared (from bend to straight and from straight to bend).

![](https://i.imgur.com/M1Yysdr.png)

[from paper](https://ieeexplore.ieee.org/ielaam/50/8720127/8684870-aam.pdf)

In [ ]:
import gdsfactory.simulation.gtidy3d as gt
import matplotlib.pyplot as plt

r, integral = gt.modes.sweep_bend_loss(wavelength=1.55, wg_width=0.5, wg_thickness=0.22, slab_thickness=0.0, ncore=gt.modes.si, nclad=gt.modes.sio2, bend_radius_min = 2.0, bend_radius_max = 5, steps = 4, mode_index = 0)

plt.plot(r, integral, ".")
plt.xlabel("bend radius (um)")
plt.ylabel("Transmission")
plt.show()